## **1. Install and import bibraries**


In [1]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

import os
import json
import torch
import random
import evaluate
import numpy as np
import pandas as pd
import torch.nn as nn
from torchcrf import CRF
from datasets import Dataset
from transformers import Trainer
from datasets import load_dataset
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from transformers import AutoTokenizer, TrainingArguments
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf


from IPython.display import clear_output
clear_output()

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def format_parameters(num_params):
    return "{:,}".format(num_params)

In [4]:
label2id = {
    'O': 0,
    'B-BODY': 1,
    'I-BODY': 2,
    'B-SYMP': 3,
    'I-SYMP': 4,
    'B-INST': 5,
    'I-INST': 6,
    'B-EXAM': 7,
    'I-EXAM': 8,
    'B-CHEM': 9,
    'I-CHEM': 10,
    'B-DISE': 11,
    'I-DISE': 12,
    'B-DRUG': 13,
    'I-DRUG': 14,
    'B-SUPP': 15,
    'I-SUPP': 16,
    'B-TREAT': 17,
    'I-TREAT': 18,
    'B-TIME': 19,
    'I-TIME': 20
}

id2label = {
    0: 'O',
    1: 'B-BODY',
    2: 'I-BODY',
    3: 'B-SYMP',
    4: 'I-SYMP',
    5: 'B-INST',
    6: 'I-INST',
    7: 'B-EXAM',
    8: 'I-EXAM',
    9: 'B-CHEM',
    10: 'I-CHEM',
    11: 'B-DISE',
    12: 'I-DISE',
    13: 'B-DRUG',
    14: 'I-DRUG',
    15: 'B-SUPP',
    16: 'I-SUPP',
    17: 'B-TREAT',
    18: 'I-TREAT',
    19: 'B-TIME',
    20: 'I-TIME'
}

In [5]:
# Mamba 的 config 類引用了這個詞: https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/config_mamba.py
@dataclass
class MambaConfig:
    d_model: int = 2560
    n_layer: int = 64
    vocab_size: int = 50277
    ssm_cfg: dict = field(default_factory=dict)
    rms_norm: bool = True
    residual_in_fp32: bool = True
    fused_add_norm: bool = True
    # pad_vocab_size_multiple: int = 8
    pad_vocab_size_multiple: int = 16
    tie_embeddings: bool = True

    def to_json_string(self):
        return json.dumps(asdict(self))

    def to_dict(self):
        return asdict(self)

In [6]:
# 用於分類的頭部類別的定義
class MambaClassificationHead(nn.Module):
    def __init__(self, d_model, num_classes, **kwargs):
        super(MambaClassificationHead, self).__init__()
        # 使用線性圖層根據輸入執行分類，該輸入的大小d_model且num_classes需要排序。
        self.classification_head = nn.Linear(d_model, num_classes, **kwargs)

    def forward(self, hidden_states):
        return self.classification_head(hidden_states)

In [7]:
class MambaTextClassification(MambaLMHeadModel):
    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

        # 使用 MambaClassificationHead 創建一個分類器，輸入大小為 d_model，類號為 len(id2label)。
        self.classification_head = MambaClassificationHead(d_model=config.d_model, num_classes=len(label2id))

        del self.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None):
        # 通過原生模型發送input_ids以接收hidden_states。
        hidden_states = self.backbone(input_ids)

        # 取二維hidden_states的平均值，創建具有代表性的 [CLS] 特徵
        mean_hidden_states = hidden_states.mean(dim=1)

        # 將mean_hidden_states通過分類器的頂部來接收logits。
        logits = self.classification_head(mean_hidden_states)

        if labels is None:
          ClassificationOutput = namedtuple("ClassificationOutput", ["logits"])
          return ClassificationOutput(logits=logits)
        else:
          ClassificationOutput = namedtuple("ClassificationOutput", ["loss", "logits"])

          # 使用 CrossEntropyLoss 損失函數計算損失。
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(logits, labels)

          return ClassificationOutput(loss=loss, logits=logits)

    def predict(self, text, tokenizer, id2label=None):
        input_ids = torch.tensor(tokenizer(text)['input_ids'], device=device)[None] # device = 'cuda'
        with torch.no_grad():
          logits = self.forward(input_ids).logits[0]
          label = np.argmax(logits.cpu().numpy())

        if id2label is not None:
          return id2label[label]
        else:
          return label

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        # 從之前訓練的模型載入配置。
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)

        # 從配置中初始化模型，並將其傳輸到所需的設備和數據類型。
        model = cls(config, device=device, dtype=dtype, **kwargs)

        # 載入以前訓練的模型狀態。
        model_state_dict = load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype)
        model.load_state_dict(model_state_dict, strict=False)

        # 列印出新初始化的嵌入參數。
        print("Newly initialized embedding:", set(model.state_dict().keys()) - set(model_state_dict.keys()))
        return model

In [8]:
# # 從先前訓練的模型載入 Mamba 模型。
model = MambaTextClassification.from_pretrained("state-spaces/mamba-130m")
# model = MambaLMHeadModel.from_pretrained("state-spaces/mamba-130m")
model.to(device)

# 從 gpt-neox-20b 模型載入 Mamba 模型的分詞器。
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer = AutoTokenizer.from_pretrained("p208p2002/llama-traditional-chinese-120M")

# 從 gpt-neox-20b 模型載入 Mamba 模型的分詞器。
tokenizer.pad_token_id = tokenizer.eos_token_id

Newly initialized embedding: {'classification_head.classification_head.bias', 'classification_head.classification_head.weight'}


In [9]:
model

MambaTextClassification(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (classification_head): MambaClassificationHead(
    (classification_head): Linear(in_features=768, out_features=21, bias=True)
  )
)

In [10]:
model_params = count_parameters(model)

In [11]:
print(f"mamba_model parameters: {format_parameters(model_params)}")

mamba_model parameters: 129,151,509


In [12]:
# class Mamba_CRF(nn.Module):
#     def __init__(self, tag_to_ix, hidden_dim = 50280):
#         super(Mamba_CRF, self).__init__()
#         self.tag_to_ix = tag_to_ix
#         self.tagset_size = len(tag_to_ix)
#         self.hidden_dim = hidden_dim

#         # self.bert = BertModel.from_pretrained('bert-base-chinese')
#         self.mamba = MambaLMHeadModel.from_pretrained("state-spaces/mamba-130m").to(device)
#         self.linear = nn.Linear(hidden_dim, self.tagset_size)
#         self.crf = CRF(self.tagset_size, batch_first=True)

#     def _get_features(self, sentence):
#         with torch.no_grad():
#           embeds, _  = self.bert(sentence)
#         enc, _ = self.lstm(embeds)
#         enc = self.dropout(enc)
#         feats = self.linear(enc)
#         return feats

#     def forward(self, sentence, tags, mask, is_test=False):
#         emissions = self._get_features(sentence)
#         if not is_test: # Training，return loss
#             loss=-self.crf.forward(emissions, tags, mask, reduction='mean')
#             return loss
#         else: # Testing，return decoding
#             decode=self.crf.decode(emissions, mask)
#             return decode

In [13]:
# VOCAB = ('<PAD>', '[CLS]', '[SEP]', 'O', 'B-BODY', 'I-BODY',
#          'B-SYMP', 'I-SYMP', 'B-INST', 'I-INST', 'B-EXAM', 'I-EXAM',
#          'B-CHEM', 'I-CHEM','B-DISE', 'I-DISE', 'B-DRUG', 'I-DRUG',
#          'B-SUPP', 'I-SUPP', 'B-TREAT', 'I-TREAT', 'B-TIME', 'I-TIME')

# tag2idx = {tag: idx for idx, tag in enumerate(VOCAB)}

# model = Mamba_CRF(tag2idx)